In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
import undetected_chromedriver as uc
import time
import pandas as pd

def configurar_driver():
    brave_path = "/usr/bin/brave-browser"  # Substitua pelo caminho real do Brave no seu sistema

    # Configuração das opções do Chrome para usar o Brave
    options = uc.ChromeOptions()
    options.binary_location = brave_path

    # Inicializar o ChromeDriver com as opções configuradas
    driver = uc.Chrome(options=options)
    return driver

def navegar_para_site(driver, url):
    # Navegar para o link especificado
    driver.get(url)

    # Esperar até que um elemento específico esteja presente na página
    # Ajuste o seletor do elemento conforme necessário
    try:
        elemento_especifico = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'h5'))
        )
        print("Página carregada com sucesso")
    except Exception as e:
        print(f"Erro ao carregar a página: {e}")
        driver.quit()

def clicar_no_botao(driver):
    try:
        # Esperar até que o ícone esteja presente
        icone = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/app-root/div/div/div/app-cabecalho-compra-acesso-publico/app-cabecalho-compra/div[4]/div[2]/div/app-botoes-cabecalho-compra/span/app-botao-mensagens-da-compra/span/app-botao-icone/span/button'))
        )
        # Encontrar o elemento pai <button> do ícone
        icone.click()
        print("Botão clicado com sucesso")
    except Exception as e:
        print(f"Erro ao clicar no botão: {e}")

def capturar_mensagens_pagina(driver, url, pagina_atual):
    tentativas = 0
    max_tentativas = 3  # Número máximo de tentativas

    while tentativas < max_tentativas:
        time.sleep(10)
        try:
            # Esperar até que os elementos de mensagens estejam presentes
            mensagens = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'cp-mensagens-compra'))
            )
            lista_mensagens = []

            # Função auxiliar para buscar elementos de forma segura
            def buscar_texto(elemento, by, value):
                try:
                    return elemento.find_element(by, value).text
                except NoSuchElementException:
                    return None

            for mensagem in mensagens:
                remetente = buscar_texto(mensagem, By.CLASS_NAME, 'mensagens-remetente')
                texto = buscar_texto(mensagem, By.CLASS_NAME, 'mensagens-texto')
                item = buscar_texto(mensagem, By.CLASS_NAME, 'mensagens-item')
                data_hora = buscar_texto(mensagem, By.CLASS_NAME, 'mensagens-data')

                dados_mensagem = {
                    'remetente': remetente,
                    'texto': texto,
                    'item': item,
                    'data_hora': data_hora
                }

                lista_mensagens.append(dados_mensagem)

                # Imprimir cada mensagem capturada
                print(f"Remetente: {dados_mensagem['remetente']}")
                if dados_mensagem['item']:
                    print(f"Item: {dados_mensagem['item']}")
                print(f"Texto: {dados_mensagem['texto']}")
                if dados_mensagem['data_hora']:
                    print(f"Data/Hora: {dados_mensagem['data_hora']}")
                print("-" * 40)

            return lista_mensagens

        except TimeoutException:
            tentativas += 1
            print(f"TimeoutException na página {pagina_atual}. Tentativa {tentativas} de {max_tentativas}...")
            if tentativas >= max_tentativas:
                print(f"Máximo de tentativas alcançado na página {pagina_atual}. Pulando esta página.")
                return []

def navegar_para_proxima_pagina(driver, pagina_atual):
    try:
        botao_proxima_pagina = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.p-paginator-next"))
        )

        # Verificar se o botão "Próxima página" está clicável
        if botao_proxima_pagina and botao_proxima_pagina.is_enabled():
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.p-paginator-next"))
            )
            # Forçar o clique via JavaScript
            driver.execute_script("arguments[0].click();", botao_proxima_pagina)
            print(f"Página {pagina_atual} processada")
            return True
        else:
            # Tentar clicar no botão da última página
            try:
                botao_pagina_atual = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, f"//button[@aria-label='Page {pagina_atual + 1}']"))
                )
                driver.execute_script("arguments[0].click();", botao_pagina_atual)
                print(f"Botão da página {pagina_atual + 1} clicado com sucesso")
                return True
            except Exception as e:
                print(f"Erro ao clicar no botão da página {pagina_atual + 1}: {e}")
                return False
    except NoSuchElementException:
        print("Botão 'Próxima página' não encontrado.")
        return False
    except Exception as e:
        print(f"Erro ao clicar no botão 'Próxima página': {e}")
        return False

if __name__ == "__main__":
    driver = configurar_driver()
    url = 'https://cnetmobile.estaleiro.serpro.gov.br/comprasnet-web/public/compras/acompanhamento-compra?compra=99019205900062024'
    navegar_para_site(driver, url)
    clicar_no_botao(driver)

    # Lista para armazenar todas as mensagens de todas as páginas
    todas_mensagens = []
    pagina_atual = 1

    # Capturar mensagens de todas as páginas
    while True:
        todas_mensagens.extend(capturar_mensagens_pagina(driver, url, pagina_atual))
        time.sleep(1)
        pagina_atual += 1
        # Verificar se há uma próxima página e navegar
        if not navegar_para_proxima_pagina(driver, pagina_atual):
            break

    # Converter lista de dicionários em DataFrame
    df = pd.DataFrame(todas_mensagens)

    # Salvar DataFrame em um arquivo Excel
    df.to_excel('mensagens_chat.xlsx', index=False, engine='openpyxl')

    print("Dados exportados para mensagens_chat.xlsx")

    driver.close()


Página carregada com sucesso
Botão clicado com sucesso
Remetente: Mensagem do Pregoeiro
Item: Item 6
Texto: O item 6 está na etapa de habilitação de fornecedores no período de intenção de recursos, com acréscimo de 10 minutos a partir de agora - até 18/06/2024 15:19:59.
Data/Hora: Enviada em 18/06/2024 às 15:09:59h
----------------------------------------
Remetente: Mensagem do Pregoeiro
Item: Item 3
Texto: O item 3 está na etapa de habilitação de fornecedores no período de intenção de recursos, com acréscimo de 10 minutos a partir de agora - até 18/06/2024 15:19:36.
Data/Hora: Enviada em 18/06/2024 às 15:09:36h
----------------------------------------
Remetente: Mensagem do Pregoeiro
Item: Item 16
Texto: O item 16 está na etapa de habilitação de fornecedores no período de intenção de recursos, com acréscimo de 10 minutos a partir de agora - até 18/06/2024 15:11:15.
Data/Hora: Enviada em 18/06/2024 às 15:01:15h
----------------------------------------
Remetente: Mensagem do Pregoeiro
I